In [1]:
# Usual Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn import preprocessing
import audioread
from tqdm import tqdm

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
# warnings.filterwarnings('ignore')

In [2]:
import os
general_path = '../input/gtzan-dataset-music-genre-classification/Data'
print(list(os.listdir(f'{general_path}/genres_original/')))

['disco', 'metal', 'reggae', 'blues', 'rock', 'classical', 'jazz', 'hiphop', 'country', 'pop']


In [3]:
def extract_features(file_name):
    # Load audio file
    try:
        y, sr = librosa.load(file_name)

        # Extract features
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

        # Aggregate features
        features = {
            'mfccs_mean': np.mean(mfccs, axis=1),
            'spectral_centroid_mean': np.mean(spectral_centroid),
            'spectral_bandwidth_mean': np.mean(spectral_bandwidth),
            'spectral_rolloff_mean': np.mean(spectral_rolloff),
            'zero_crossing_rate_mean': np.mean(zero_crossing_rate)
        }

        # Flatten the dictionary
        flattened_features = {}
        for key, value in features.items():
            if isinstance(value, np.ndarray):
                for i, v in enumerate(value):
                    flattened_features[f'{key}_{i}'] = v
            else:
                flattened_features[key] = value

        return flattened_features
    except:
        return None

In [4]:
def process_dataset(directory):
    # List to hold all extracted features
    all_features = []

    # Get total number of files to process for tqdm
    total_files = sum([len(files) for r, d, files in os.walk(directory) if any(file.endswith('.wav') for file in files)])
    print(f"Total number of .wav files to process: {total_files}")

    # Traverse all files and folders in the directory with progress bar
    with tqdm(total=total_files, desc="Processing files") as pbar:
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith('.wav'):  # Adjust this if your files have different extensions
                    file_path = os.path.join(root, file)
                    features = extract_features(file_path)
                    if features:  # Only add if features extraction was successful
                        features['file_name'] = file_path  # Store full path to identify file
                        all_features.append(features)
                    pbar.update(1)
    
    # Convert to DataFrame
    features_df = pd.DataFrame(all_features)
    return features_df

In [ ]:
features_df = process_dataset(f'{general_path}/genres_original/')
features_df.to_csv('extracted_features.csv', index=False)

Total number of .wav files to process: 1000


Processing files:  20%|█▉        | 195/1000 [01:26<04:33,  2.94it/s]

In [ ]:
features_df